In [26]:
import numpy as np 
import pandas as pd
import sys
import csv
import glob
import folium
from folium.plugins import HeatMap

from event_prediction_data import *

In [23]:
path = "C:/Users/huigyu/workspace/BikeSharingSystems_rnd/03.Data/use_historical/20191210.csv"
data = pd.read_csv(path)
dir_path = "C:/Users/huigyu/workspace/BikeSharingSystems_rnd/03.Data/use_historical/*.csv"

In [24]:
station_list = []
data_dict = {}
location = {}
day = 1
max_val = 0

for i, file in enumerate(sorted(glob.glob(dir_path))):
    if i >= day:
        break
    file = open(file, "r", encoding="utf-8")
    csv_reader = csv.reader(file)
    
    for j, line in enumerate(csv_reader):
        station = line[2]  
        lat, lon = line[10], line[11]
        
        if not station_list.__contains__(station):
            station_list.append(station)
            
        if not data_dict.__contains__(station):
            data_dict[station] = [int(line[7])]
        else:
            data_dict[station].append(int(line[7]))
        
        if not location.__contains__(station):
            location[station] = [float(lat), float(lon)]

            
            
    list_max = max([len(data_dict[d]) for d in data_dict])
    
    for d in data_dict:
        len_ = len(data_dict[d])
        if len_ < list_max:
            data_dict.pop(d)
            station_list.remove(d)
            location.pop(d)

In [4]:
# location plot

average_lat = sum(lat for lat, long in location.values()) / len(location)
average_long = sum(long for lat, long in location.values()) / len(location)


In [ ]:
location

In [9]:
map = folium.Map(location=[average_lat, average_long],
                 zoom_start=12,
                 tiles = "CartoDB positron")

for station, (lat, long) in location.items():
    folium.CircleMarker([lat, long],
                        popup=station,
                        radius=2,
                        color='red',
                        fill=False).add_to(map)



In [16]:
# location heatmap

hmap = folium.Map(location=[average_lat, average_long],
                  zoom_start=12,
                  tiles="CartoDB positron")

heat_map_data = [lc for lc in location.values()]
HeatMap(heat_map_data,
        radius=15,
        blur=10).add_to(hmap)

hmap

In [20]:

boundary = [0.00,0.20,0.40,0.60,0.80,sys.maxsize/100]
(data_dict, station_list, station_total_dock, max_val) = historical_station_logs(10)
data_list_categorical = []

for lock, station_name in enumerate(data_dict):
    cate_temp = []
    for x in data_dict[station_name]:
        per = x / station_total_dock[station_name]
        
        for ca in range(1,len(boundary)):
            if int(per*100) in range(int(boundary[ca-1]*100),int(boundary[ca]*100)):
                cate_temp.append(ca-1)
                break
                
data_list_categorical.append(cate_temp)

Number of Stations:  0a	|████-----------------------------------------| 9.1% 0.00/0.00s


IndexError: list index out of range

In [ ]:
def multi_layer_map_cate(data_list, location, category_colors, time_steps):

    groups = {}
    for i in range(time_steps):
        group = FeatureGroupSubGroup(map, f"time_period_{i}")
        map.add_child(group)
        groups[i] = group

    for time_step, data in enumerate(time_data_list):
        for station, category in zip(location.keys(), data):
            color = category_colors[category]
            folium.CircleMarker(location[station],
                                radius=2,
                                color=color,
                                fill=True).add_to(groups[time_step])

    folium.LayerControl(collapsed=False).add_to(map)